# ***멀티모달 흉내내기***
- 하나의 AI에서 영상, 음성, 텍스트를 통합하기 위해서는 아래의 과정이 필요합니다
  - 영상의 객체탐지 정보를 언어모델에서 사용하는 숫자(토큰) 정보로 변환
  - 음성의 텍스트 정보를 언어모델에서 사용하는 숫자(토큰) 정보로 변환
  - 언어모델의 토큰으로 변환된 영상, 음성정보, 우리의 프롬프트를 이용해 다음 문장 생성
- 우리가 흉내낼 멀티모달은 아래와 같습니다
  - 영상의 객체탐지 정보를 우리의 언어로 변환
  - 음성의 텍스트 정보를 우리의 언어로 변환
  - 우리의 언어로변환된 영상, 음성정보와 우리의 프롬프트를 언어모델의 토큰으로 변환
  - 변환된 토큰으로 다음 문장 생성

## ***영상 객체 탐지 정보를 텍스트로 만들기***

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

image_model = YOLO("yolov8n.pt")

def detect_objects_with_interval(video_path, interval=1.0):  
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    t_secs = np.arange(0, duration, interval)

    all_detections = {}
    for t_sec in t_secs:
        frame_idx = int(t_sec * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)

        ret, frame = cap.read()
        if not ret:
            all_detections[t_sec] = []
            continue

        results = image_model(frame)[0]

        detections = []
        for box in results.boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            detections.append({
                "class": image_model.names[int(box.cls[0])],
                "bbox": [int(x1), int(y1), int(x2), int(y2)],
                "conf": float(box.conf[0])
            })

        all_detections[t_sec] = detections

    cap.release()
    return all_detections


def image_to_text(video_path, interval=1.0):
    detections = detect_objects_with_interval(video_path, interval)
    lines = []

    for t_sec, objs in detections.items():
        if not objs:
            lines.append(f"{t_sec:.1f}초에는 탐지된 객체가 없었습니다.")
            continue

        descriptions = []
        for obj in objs:
            cls = obj['class']
            x1, y1, x2, y2 = obj['bbox']
            descriptions.append(f"좌표 {x1},{y1}부터 {x2},{y2} 사이에 {cls}이(가) 있었습니다")

        line = f"{t_sec:.1f}초에는 " + " 그리고 ".join(descriptions) + "."
        lines.append(line)

    return "\n".join(lines)


video_path = "movie.mp4"
interval = 1 # 1초 간격으로 영상 샘플링
image_text = image_to_text(video_path, interval)
print(image_text)


## ***음성을 타임라인별로 텍스트화 하기***

In [ ]:
import whisper

voice_model = whisper.load_model("small")

def transcribe_audio(video_path):    
    result = voice_model.transcribe(video_path)

    timeline = []
    for seg in result["segments"]:
        timeline.append({
            "start": seg["start"],
            "end": seg["end"],
            "text": seg["text"]
        })

    return timeline

def audio_to_text(audio_timeline):
    lines = []
    for seg in audio_timeline:
        lines.append(f"{seg['start']:.1f}s 부터 {seg['end']:.1f}s 사이에 담긴 음성은 {seg["text"]} 입니다")
        
    return " ".join(lines)


audio_timeline = transcribe_audio(video_path)
audio_text = audio_to_text(audio_timeline)
audio_text


## ***이미지와 음성과 프롬프트를 텍스트로 통합***

In [ ]:
prompt = f"객체 탐지 정보는 아래와 같습니다\n{image_text}\n추출된 음성 정보는 아래와 같습니다\n{audio_text}\n주어진 객체와 음성 정보를 기반으로 동영상을 설명해보세요"
print(prompt)

## ***통합 프롬프트를 언어모델의 입력으로 실행***

In [ ]:
import ollama

image_model='llama3.2:1b'

messages = [{
        'role': 'user',
        'content': prompt
    }]

response = ollama.chat(
    model=image_model,
    messages=messages,
    options={
        'temperature': 0.8
    })
content = response['message']['content']

print(content)

## ***동영상으로 시작하여 언어모델에 한번에 연결하기***
- 새로운 동영상을 만들고 언어모델까지 한번에 연결해봅니다

In [ ]:
import ollama

video_path = "1.mp4"

image_text = image_to_text(video_path, 1)
audio_timeline = transcribe_audio(video_path)
audio_text = audio_to_text(audio_timeline)
prompt = f"객체 탐지 정보는 아래와 같습니다\n{image_text}\n추출된 음성 정보는 아래와 같습니다\n{audio_text}\n주어진 객체와 음성 정보를 기반으로 동영상을 설명해보세요"

image_model='llama3.2:1b'

messages = [{
        'role': 'user',
        'content': prompt
    }]

response = ollama.chat(
    model=image_model,
    messages=messages,
    options={
        'temperature': 0.8
    })
content = response['message']['content']

print(content)
